In [1]:
# import the optimize solver Gurobi
import gurobipy as gp
from gurobipy import * 

In [2]:
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
well_lim = [150000, 200000]  # Well 1 can produces up to 150,000 barrels per day; Well 2 can produce up to 200,000 barrels per day
demand = [140000, 160000]  # New York requires 140,000 barrels per day; Los Angeles requires 160,000 barrels per day

In [4]:
lim = [150000, 200000, 0, 0, 140000, 160000]

In [5]:
# Set each node
nodes = ['well1', 'well2', 'mobile', 'galveston', 'NY', 'LA']

nodes_lim = {}
for i in range(len(nodes)):
    nodes_lim[nodes[i]] = lim[i]

# Set arcs, cost from orign to destination

arcs, cost = gp.multidict({
    ('well1', 'mobile'):   3530,
    ('well1', 'galveston'):  3420,
    ('well1', 'NY'): 3920,
    ('well1',  'LA'):  3710,
    ('well2',  'mobile'): 3650,
    ('well2',  'galveston'):  3580,
    ('well2', 'NY'):   3840,
    ('well2', 'LA'):  3620,
    ('mobile', 'galveston'):  1200,
    ('mobile',  'NY'):   3680,
    ('mobile',  'LA'): 3740,
    ('galveston',  'mobile'):  1500,
    ('galveston',  'NY'):   3570,
    ('galveston',  'LA'): 3650,
    ('NY',  'LA'):  4120,
    ('LA',  'NY'):   3980 })

In [6]:
m.update()
print(nodes_lim)

{'well1': 150000, 'well2': 200000, 'mobile': 0, 'galveston': 0, 'NY': 140000, 'LA': 160000}


In [7]:
# Create variables
flow = m.addVars(arcs, obj = cost, name="flow")

In [8]:
# Flow-conservation constraints
m.addConstrs((flow.sum(i, '*') <= nodes_lim[i] for i in nodes[:2] ),"c1")
m.addConstrs(((flow.sum('*', i) - flow.sum(i, '*')) == nodes_lim[i] for i in nodes[4:] ),"c2")
m.addConstrs(((flow.sum('*', i) - flow.sum(i, '*')) == nodes_lim[i] for i in nodes[2:4]),"c3")

{'mobile': <gurobi.Constr *Awaiting Model Update*>,
 'galveston': <gurobi.Constr *Awaiting Model Update*>}

In [9]:
# Compute optimal solution
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 6 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x98434d04
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 2e+05]
Presolve time: 0.11s
Presolved: 6 rows, 16 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.000000e+05   0.000000e+00      0s
       5    1.1248000e+09   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.13 seconds
Optimal objective  1.124800000e+09


In [10]:
m.printAttr('X')


    Variable            X 
-------------------------
flow[well1,NY]       100000 
flow[well2,NY]        40000 
flow[well2,LA]       160000 


In [11]:
# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', flow)
    for i, j in arcs:
        if solution[i,j] > 0:
            print('%s -> %s : %g' % (i, j, solution[i,j]))

well1 -> NY : 100000
well2 -> NY : 40000
well2 -> LA : 160000


In [12]:
# Get the Optimal Objective Value
m.ObjVal

1124800000.0